In [1]:
import pandas as pd
import os
from tqdm import tqdm
from sklearn.metrics import mean_absolute_percentage_error
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
from collections import defaultdict
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
import datetime as dt
from sklearn.model_selection import GridSearchCV

In [2]:
FILES_FOLDER = {
    "clean_data": [
        "clean_data/building.csv",
        "clean_data/meters.csv",
        "clean_data/weather.csv"
        #"data_for_data_challenge/X_test_data.csv", # The dataset test will be released one hour before the end of the Data Challenge
    ]
}


def load_data(folder, data_dir, dict_files=FILES_FOLDER):
    files = dict_files[folder]
    dataframes = []
    print(":: Start loading data")
    for name_file in tqdm(files):
        dataframe = pd.read_csv(os.path.join(data_dir, name_file))
        dataframes.append(dataframe)
    return dataframes


building, meters, weather = load_data("clean_data", "data_for_data_challenge/")

:: Start loading data


100%|██████████| 3/3 [00:00<00:00,  4.36it/s]


ENLEVER LE SITE 11
ENLEVER SEALEVEL et WIND
ENLEVER LES LIGNES NULLES ?
AJOUTER COLONNE ANNEE AVEC UNSPECIFIED
ENLEVER CATEGORIES

In [3]:
sorted_meters = meters.sort_values(by = 'timestamp')
sorted_weather = weather.sort_values(by = 'timestamp').drop(['Unnamed: 0','index'], axis = 1)
sorted_meters = sorted_meters[['timestamp','building_id','meter','meter_reading']]
sorted_meters = sorted_meters.join(building[['building_id','site_id','primary_use','sub_primary_use','square_feet','year_built']].set_index('building_id'), on = 'building_id')
sorted_to_merge = sorted_meters.copy()
merged_data = sorted_to_merge.merge(sorted_weather, how='inner', left_on=['timestamp', 'site_id'], right_on=['timestamp', 'site_id'])

In [4]:
#encoding of years
def extract_years(str):
    return dt.datetime.strptime(str,"%Y-%m-%d %H:%M:%S").year
extract_years = np.vectorize(extract_years)
years = extract_years(merged_data.timestamp)
merged_data['YEAR'] =  years-min(years) 
#encoding of days
def extract_day(str):
    return dt.datetime.strptime(str,"%Y-%m-%d %H:%M:%S").timetuple().tm_yday
extract_day = np.vectorize(extract_day)
days = extract_day(merged_data.timestamp)
merged_data['day'] =  days
#encoding of 
def extract_hour(str):
    return dt.datetime.strptime(str,"%Y-%m-%d %H:%M:%S").hour
extract_hour = np.vectorize(extract_hour)
hours = extract_hour(merged_data.timestamp)
merged_data['hour'] =  hours
#merged_data.drop(['timestamp'], axis = 1)
merged_data['Day/Night'] = merged_data['hour'].where((merged_data['hour']>=7) & (merged_data['hour']<=20),'Daylight')
merged_data['Day/Night'] = merged_data['Day/Night'].where((merged_data['Day/Night'] == 'Daylight'),'Night')
merged_data = pd.concat([merged_data,pd.get_dummies(merged_data['Day/Night'])], axis = 1)
merged_data = merged_data.drop(['Day/Night'], axis = 1)
def extract_weekday(str):
    return dt.datetime.strptime(str,"%Y-%m-%d %H:%M:%S").weekday()
extract_weekday = np.vectorize(extract_weekday)
merged_data['weekday']=extract_weekday(merged_data.timestamp)
merged_data['weekday'] = merged_data['weekday'].where(merged_data['weekday']>=5,'week_in')
merged_data['weekday'] = merged_data['weekday'].where(merged_data['weekday']=='week_in','weekend')
merged_data = pd.concat([merged_data,pd.get_dummies(merged_data['weekday'])], axis = 1).drop(['weekday'], axis = 1)

In [5]:
merged_data

,timestamp,building_id,meter,meter_reading,site_id,primary_use,sub_primary_use,square_feet,year_built,air_temperature,...,sea_level_pressure,wind_direction,wind_speed,YEAR,day,hour,Daylight,Night,week_in,weekend
0,2016-01-01 00:00:00,198,0,197.8700,2.0,Office,Office,163959.0,2004.0,15.6,...,1015.5,270.0,3.6,0,1,0,1,0,1,0
1,2016-01-01 00:00:00,239,3,67.4063,2.0,Lodging/residential,Dormitory,106440.0,1963.0,15.6,...,1015.5,270.0,3.6,0,1,0,1,0,1,0
2,2016-01-01 00:00:00,276,0,115.7400,2.0,Office,Office,70837.0,2014.0,15.6,...,1015.5,270.0,3.6,0,1,0,1,0,1,0
3,2016-01-01 00:00:00,234,0,63.0700,2.0,Education,College Classroom,179167.0,1971.0,15.6,...,1015.5,270.0,3.6,0,1,0,1,0,1,0
4,2016-01-01 00:00:00,191,0,54.8500,2.0,Education,College Classroom,78268.0,1964.0,15.6,...,1015.5,270.0,3.6,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872589,2017-06-30 23:00:00,46,0,35.4351,0.0,Retail,Retail,9045.0,2016.0,25.6,...,1018.5,0.0,0.0,1,181,23,1,0,1,0
872590,2017-06-30 23:00:00,1031,3,5.0112,11.0,Education,Academic,93206.0,NaN,23.1,...,1008.0,180.0,3.6,1,181,23,1,0,1,0
872591,2017-06-30 23:00:00,658,0,0.0003,5.0,Entertainment/public assembly,Community Center,31075.0,1976.0,14.0,...,NaN,310.0,3.1,1,181,23,1,0,1,0
872592,2017-06-30 23:00:00,739,0,0.0074,5.0,Education,Primary/Secondary Classroom,137046.0,1966.0,14.0,...,NaN,310.0,3.1,1,181,23,1,0,1,0


In [6]:
new_temp = (merged_data[merged_data['site_id']==14]['air_temperature']-32)/1.8
index_14 = merged_data[merged_data['site_id']==14].index
merged_data.loc[index_14,'air_temperature']=new_temp

dict_site_id = {
    0 : 'US',
    1 : 'EU',
    2 : 'US',
    3 : 'US',
    4 : 'US',
    5 : 'EU',
    6 : 'US',
    7 : 'US',
    8 : '',
    9 : 'US',
    10 : '',
    11 : '',
    12 : 'EU',
    13 : 'US',
    14 : '',
    15 : 'US'
} 
merged_data['US/EU'] = merged_data['site_id'].map(dict_site_id)
a = pd.get_dummies(merged_data['US/EU'])
merged_data = pd.concat([merged_data,a], axis = 1).drop(['US/EU'], axis = 1)

a = pd.get_dummies(merged_data['primary_use'])
label = merged_data['primary_use'].unique()
label.sort()
a.columns = label
merged_data = pd.concat([merged_data,a], axis = 1).drop(['primary_use'], axis = 1)

a = pd.get_dummies(merged_data['site_id'])
label = merged_data['site_id'].unique()
label.sort()
a.columns = label
merged_data = pd.concat([merged_data,a], axis = 1).drop(['site_id'], axis = 1)

a = pd.get_dummies(merged_data['meter'])
a.columns = ['Meter 0','Meter 1','Meter 2','Meter 3']
merged_data = pd.concat([merged_data,a], axis = 1).drop(['meter'], axis = 1)

In [9]:
merged_data.drop(['sub_primary_use','sea_level_pressure','wind_speed','wind_direction','timestamp','year_built'], axis = 1)

,building_id,meter_reading,square_feet,air_temperature,dew_temperature,YEAR,day,hour,Daylight,Night,...,10.0,11.0,12.0,13.0,14.0,15.0,Meter 0,Meter 1,Meter 2,Meter 3
0,198,197.8700,163959.0,15.6,-5.6,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
1,239,67.4063,106440.0,15.6,-5.6,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,276,115.7400,70837.0,15.6,-5.6,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,234,63.0700,179167.0,15.6,-5.6,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,191,54.8500,78268.0,15.6,-5.6,0,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872589,46,35.4351,9045.0,25.6,22.8,1,181,23,1,0,...,0,0,0,0,0,0,1,0,0,0
872590,1031,5.0112,93206.0,23.1,20.1,1,181,23,1,0,...,0,1,0,0,0,0,0,0,0,1
872591,658,0.0003,31075.0,14.0,12.0,1,181,23,1,0,...,0,0,0,0,0,0,1,0,0,0
872592,739,0.0074,137046.0,14.0,12.0,1,181,23,1,0,...,0,0,0,0,0,0,1,0,0,0
